In [5]:
# %% 📝 Souring Path
import sys, os
SRC_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "src"))
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
print(f"✅ SRC Path: {SRC_PATH}")


✅ SRC Path: /home/prashant-agrawal/projects/netflix_talk2data/src


In [6]:
# Python imports & libraries
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from qdrant_client.http.models import Range
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PayloadSchemaType, FieldCondition, MatchValue, Filter
from typing import Dict, Union, Any
import pandas as pd
import re
import json

# 🚀 Import your utility loaders
from utils.qdrant_client_loader import get_qdrant_collection_name
from utils.path_config import get_base_dir, get_data_path, get_qdrant_store_path, get_schema_path

# %% 📁 Paths
BASE_DIR = get_base_dir()
DATA_PATH = get_data_path()
SCHEMA_OUTPUT_PATH = get_schema_path()
qdrant_store_path = get_qdrant_store_path()
COLLECTION_NAME = get_qdrant_collection_name()

print(f"📌 Base Dir: {BASE_DIR}")
print(f"📌 CSV Path: {DATA_PATH}")
print(f"📌 Qdrant Local Path: {qdrant_store_path}")
print(f"📌 Collection Name: {COLLECTION_NAME}")
print(f"📌 Schema Path: {SCHEMA_OUTPUT_PATH}")



Qdrant store path: /home/prashant-agrawal/projects/netflix_talk2data/src/database/qdrant_store_local_db/collection
Data path: /home/prashant-agrawal/projects/netflix_talk2data/src/Data/Enriched_Indian_Startup_Dataset.csv
Schema path: /home/prashant-agrawal/projects/netflix_talk2data/src/schema/payload_schema.json
📌 Base Dir: /home/prashant-agrawal/projects/netflix_talk2data/src
📌 CSV Path: /home/prashant-agrawal/projects/netflix_talk2data/src/Data/Enriched_Indian_Startup_Dataset.csv
📌 Qdrant Local Path: /home/prashant-agrawal/projects/netflix_talk2data/src/database/qdrant_store_local_db/collection
📌 Collection Name: indian_startups
📌 Schema Path: /home/prashant-agrawal/projects/netflix_talk2data/src/schema/payload_schema.json


In [8]:

# --- Utility: Normalize ---
def normalize_field_name(field: str) -> str:
    field = field.strip().lower()
    field = re.sub(r"[ ()/]", "_", field)
    return re.sub(r"[^a-zA-Z0-9_]", "", field)

def normalize_field_value(value) -> str:
    return str(value).strip().lower()


In [9]:
# --- 1. Load CSV ---
df = pd.read_csv(DATA_PATH)
df = df.dropna(subset=["Company Description (Long)"]).reset_index(drop=True)

# --- 2. Prepare Documents ---
def load_documents_from_df(df: pd.DataFrame):
    docs = []
    for _, row in df.iterrows():
        metadata = {
            normalize_field_name(str(k)): normalize_field_value(v)
            for k, v in row.items() if pd.notna(v)
        }
        content = "\n".join(f"{k}: {v}" for k, v in metadata.items())
        docs.append(Document(page_content=content, metadata=metadata))
    return docs

documents = load_documents_from_df(df)
print(f"✅ Loaded {len(documents)} documents.")


✅ Loaded 500 documents.
✅ Created payload indexes.
✅ Ingested 500 documents.
Number of results: 0


In [11]:

# --- 3. Connect and Recreate Collection ---
client = QdrantClient(host="localhost", port=6333)

if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(collection_name=COLLECTION_NAME)


In [12]:

# --- 4. Define Payload Schema ---
# All category fields as KEYWORD for exact matching
payload_schema = {
    "company_name": PayloadSchemaType.KEYWORD,
    "legal_entity_type": PayloadSchemaType.KEYWORD,
    "state": PayloadSchemaType.KEYWORD,
    "headquarters_city": PayloadSchemaType.KEYWORD,
    "industry_sector": PayloadSchemaType.KEYWORD,
    # All others as TEXT or INT/FLOAT as needed
}

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)


True

In [14]:

# --- 5. Create Payload Indexes for filtered fields ---
for field, schema in payload_schema.items():
    client.create_payload_index(
        collection_name=COLLECTION_NAME,
        field_name=field,
        field_schema=schema
    )

print(f"✅ Created payload indexes.")


✅ Created payload indexes.


In [15]:

# --- 6. Ingest documents into Qdrant ---
embedding_model = OpenAIEmbeddings()
qdrant = Qdrant.from_documents(
    documents=documents,
    embedding=embedding_model,
    url="http://localhost:6333",
    collection_name=COLLECTION_NAME,
)

print(f"✅ Ingested {len(documents)} documents.")


✅ Ingested 500 documents.


In [16]:

# --- 7. Test: Filter Query for 'state' = 'delhi' ---
from qdrant_client.http.models import Filter, FieldCondition, MatchValue

q_filter = Filter(must=[
    FieldCondition(key="state", match=MatchValue(value="delhi"))
])

result = client.scroll(
    collection_name=COLLECTION_NAME,
    scroll_filter=q_filter,
    limit=10,
    with_payload=True
)

print("Number of results:", len(result[0]))
for pt in result[0]:
    print(pt.payload)


Number of results: 0
